In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time

In [2]:
start_time = time.time()

de2en = {"Montag": "Monday", "Dienstag": "Tuesday", "Mittwoch": "Wednesday",
           "Donnerstag": "Thursday", "Freitag": "Friday", "Samstag": "Saturday", "Sonntag": "Sunday"}
result_list = []

# 63 - 122
R = range(63, 123)  # Year = 1900+k
#problem mit 114, 115

for k in R:
    # First load a website to just see how many matchdays there are
    prefix = 19 if k < 100 else 20
    url = ('http://www.kicker.de/news/fussball/bundesliga/spieltag/'
            '1-bundesliga/{}{:02d}-{:02d}/-1/0/spieltag.html'.format(prefix, k % 100, (k+1) % 100))
    page = requests.get(url)
    #print(url)
    
    soup = BeautifulSoup(page.text, 'html.parser')  # Get website content
    matchdaytag = "kick__v100-gameList kick__module-margin"
    matchdays = soup.findAll("div", {"class": matchdaytag})

    for matchday in range(1, 1+len(matchdays)):  # The roughly 30 matchdays through the year
        url = "https://www.kicker.de/1-bundesliga/spieltag/{}{:02d}-{:02d}/{}".format(
            prefix, k % 100, (k+1) % 100, matchday)
        #print(url)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')  # Get website content

        #print("Got Matchday " +str (matchday) + "/" + str(k) + " in: " + str(time.time() - start_time))

        matchdatestag = "kick__v100-gameList kick__module-margin"
        matchdates = soup.findAll("div", {"class": matchdatestag})
        for matchdate in matchdates:  # The weekday during which the matches take place
            weekdaytag = "kick__v100-gameList__header"
            st = matchdate.findAll("div", {"class": weekdaytag})[0].text.strip().split(",")[1]
            date = pd.to_datetime(st, format = " %d.%m.%Y").date()
            dayofweek = matchdate.findAll("div", {"class": weekdaytag})[
                0].text.strip().split(",")[0]
            matchtag = "kick__v100-gameList__gameRow"
            games = matchdate.findAll("div", {"class": matchtag})
            for game in games:  # All the games on a Friday, Saturday, ...
                teamstag = "kick__v100-gameCell__team__name"
                teams = game.findAll("div", {"class": teamstag})
                resulttag = "kick__v100-scoreBoard kick__v100-scoreBoard--standard"
                goalstag = "kick__v100-scoreBoard__scoreHolder__score"
                
                #print(str(k))
                goals = game.findAll("a", {"class": resulttag})
                #print(goals)
                if len(goals) > 0:
                    goals = goals[0]
                    #print(goals)
                    goals = goals.findAll("div", {"class": goalstag})

                    aborted = False
                    annull = False
                    try:
                        home = teams[0].text.strip()
                        guest = teams[1].text.strip()
                        goalshome = int(goals[0].text)
                        goalsguest = int(goals[1].text)
                        goalshome_halftime = int(goals[2].text)
                        goalsguest_halftime = int(goals[3].text)
                    except:
                        # Two games got aborted and repeated. We do not have a half time score for these.
                        # We do store them in the databse, but set the "aborted" flag to be true.
                        # The games are found here:
                        # https://www.kicker.de/bundesliga/spieltag/1963-64/14/0
                        # https://www.kicker.de/bundesliga/spieltag/2007-08/28/0
                        # For four more games, the half time score was not available.

                        try:
                            abortedtag = "kick__v100-scoreBoard__scoreHolder__text"
                            status = game.findAll("div", {"class": abortedtag})
                            aborted = status[0].text == "abgr."
                        except:
                            pass

                        try:
                            annultag = "kick__v100-scoreBoard__dateHolder"
                            status = game.findAll("div", {"class": abortedtag})
                            annull = status[0].text == "annull."
                        except:
                            pass

                        goalshome_halftime = pd.NA
                        goalsguest_halftime = pd.NA

                    # In 1995/96 the 3 points rule for winner was introduced.
                    pointswinner = 3 if k >= 95 else 2
                    if goalshome == "" or goalsguest == "":
                        pointshome, pointsguest = "", ""
                    elif goalshome == goalsguest:
                        pointshome, pointsguest = 1, 1
                    elif goalshome > goalsguest:
                        pointshome, pointsguest = pointswinner, 0
                    else:
                        pointshome, pointsguest = 0, pointswinner

                    entry = {'SeasonFrom': 1900+k,
                             'SeasonTo': 1901+k,
                             'Matchday': matchday,
                             'Day': de2en[dayofweek],
                             'Date': date,
                             'Time': pd.NA,
                             'Home': home,
                             'Guest': guest,
                            #'Score90': "{}:{}".format(goalshome, goalsguest),
                            #'Score45': "{}:{}".format(goalshome_halftime, goalsguest_halftime),
                             'Score90Home': goalshome,
                             'Score90Guest': goalsguest,
                             'Score45Home': goalshome_halftime,
                             'Score45Guest': goalsguest_halftime,
                             'PointsHome': pointshome,
                             'PointsGuest': pointsguest,
                            'Aborted': aborted,
                            'Annulled': annull,
                    #         'League': "{}. Bundesliga".format(league)
                            }
                    result_list.append(entry)
                    goalshome = ""
                    goalsguest = ""
                    goalshome_halftime = ""
                    goalsguest_halftime = ""
                    

    year_time = time.time() - start_time
    years_left = 59 - ((1900+k)-1963)
    time_left = year_time * years_left
    print("Finished Year " + str(1900+k) + "/" + str(1901+k) + " ("+str(years_left)+" years left) in: " + str(datetime.timedelta(seconds=year_time)), end=" | ")
    print("Estimated time left: " + str(datetime.timedelta(seconds=time_left)))
    start_time = time.time()
    #df = pd.DataFrame.from_dict(result_list)
    #df.to_csv("results/y" + str(k) + ".csv", sep=";")
    #result_list = []
df = pd.DataFrame.from_dict(result_list)
df.to_csv("results/all.csv", sep=";")

Finished Year 1963/1964 (59 years left) in: 18.2610342502594Estimated time left: 0:17:57.401021
Finished Year 1964/1965 (58 years left) in: 26.685261011123657Estimated time left: 0:25:47.745139
Finished Year 1965/1966 (57 years left) in: 48.38276505470276Estimated time left: 0:45:57.817608
Finished Year 1966/1967 (56 years left) in: 31.949623107910156Estimated time left: 0:29:49.178894
Finished Year 1967/1968 (55 years left) in: 43.414113998413086Estimated time left: 0:39:47.776270
Finished Year 1968/1969 (54 years left) in: 41.5876829624176Estimated time left: 0:37:25.734880
Finished Year 1969/1970 (53 years left) in: 44.195329904556274Estimated time left: 0:39:02.352485
Finished Year 1970/1971 (52 years left) in: 55.51817297935486Estimated time left: 0:48:06.944995
Finished Year 1971/1972 (51 years left) in: 54.26375889778137Estimated time left: 0:46:07.451704
Finished Year 1972/1973 (50 years left) in: 51.805883169174194Estimated time left: 0:43:10.294158
Finished Year 1973/1974 (49